In [1]:
'''Trains a simple deep NN on the MNIST dataset.

Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils


batch_size = 128
nb_classes = 10
nb_epoch = 20
#nb_epoch = 1

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
#X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train_fixed = []
for x in range(0,60000):
    temp=[]
    temp.append(X_train[x])
    temp=np.array(temp)
    X_train_fixed.append(temp)

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is enabled with initial size: 70.0% of memory, cuDNN 4007)


60000 train samples
10000 test samples


In [2]:
def compare(arrayA, arrayB): #compute MSE
    return (np.average(np.square(np.subtract(arrayA, arrayB))))

In [3]:
shared = Dense(512, input_shape=(784,))
sharedact= Activation('tanh')
channels = []
for x in range(0, 10):
    channelLayer = []
    channelLayer.append(Dense(784,input_shape=(512,)))
    channelLayer.append(Activation('linear'))
    channels.append(channelLayer)

models = []    
for x in range(0, 10):
    model = Sequential()
    model.add(shared)
    model.add(sharedact)
    model.add(channels[x][0])
    model.add(channels[x][1])
    model.compile(loss='mean_squared_error', optimizer=RMSprop(), metrics=['accuracy'])
    model.summary()
    models.append(model)


for x in range(0, nb_epoch):
#for x in range(0, 1):
    print(x)
    for y in range(0, 60000):

        #temp=[]
        #temp.append(X_train[y])
        #temp=np.array(temp)
        models[y_train[y]].train_on_batch(X_train_fixed[y],X_train_fixed[y])
        #models[y_train[y]].fit([temp],[temp], batch_size=1, nb_epoch=1, verbose=0) 
        
        
print('done training')


____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
dense_1 (Dense)                    (None, 512)         401920      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 512)         0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                    (None, 784)         402192      activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)          (None, 784)         0           dense_2[0][0]                    
Total params: 804112
______________________________________________________________________

In [4]:

prediction=[]
for x in range(0,10):
    print('Predicting channel %d'%(x))
    prediction.append(models[x].predict(X_train, batch_size=1, verbose=0))
    
print('Done generating predictions')

Done generating predictions


In [5]:
#convert to format similar to training data
#determine MSE for each channel and example
#determine lowest
#determine if it matches label
#print accuracy
"""
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

"""
"""
len(prediction)

10

len(prediction[0])

60000

len(prediction[0][0])

784
"""

'\nlen(prediction)\n\n10\n\nlen(prediction[0])\n\n60000\n\nlen(prediction[0][0])\n\n784\n'

In [24]:
pred2=[]
for x in range(0,len(prediction[0])):
    temp = []
    for y in range(0,10):
        z=(compare(prediction[y][x],X_train_fixed[x]))
        temp.append(z)
    pred2.append(np.argmin(temp))
    
correct=0.0
for x in range(0,len(prediction[0])):
    if(pred2[x]==y_train[x]):
        correct+=1

print (correct)
print (float(correct/60000))

54706.0
1.09412


In [27]:
print (correct)
print (float(correct/60000))

54706.0
0.911766666667


In [25]:
pred2

[3,
 0,
 4,
 1,
 9,
 2,
 1,
 3,
 1,
 4,
 3,
 5,
 3,
 6,
 1,
 7,
 2,
 8,
 6,
 9,
 4,
 0,
 9,
 1,
 3,
 2,
 4,
 3,
 2,
 7,
 3,
 8,
 6,
 9,
 0,
 5,
 6,
 0,
 7,
 6,
 1,
 8,
 7,
 9,
 3,
 9,
 8,
 5,
 3,
 3,
 3,
 0,
 7,
 7,
 9,
 8,
 0,
 9,
 4,
 1,
 4,
 4,
 6,
 0,
 4,
 5,
 6,
 1,
 0,
 0,
 8,
 7,
 1,
 6,
 3,
 0,
 2,
 1,
 1,
 9,
 8,
 0,
 2,
 6,
 7,
 8,
 3,
 9,
 0,
 4,
 6,
 7,
 4,
 6,
 8,
 0,
 7,
 8,
 3,
 1,
 9,
 7,
 1,
 7,
 1,
 1,
 6,
 3,
 0,
 2,
 9,
 3,
 1,
 1,
 0,
 4,
 9,
 2,
 0,
 0,
 2,
 0,
 2,
 7,
 1,
 8,
 6,
 4,
 1,
 6,
 3,
 4,
 1,
 9,
 3,
 3,
 3,
 8,
 3,
 4,
 3,
 7,
 4,
 2,
 8,
 5,
 8,
 6,
 9,
 3,
 4,
 6,
 1,
 9,
 9,
 6,
 0,
 3,
 9,
 2,
 8,
 2,
 9,
 4,
 4,
 6,
 4,
 9,
 7,
 0,
 9,
 2,
 9,
 5,
 1,
 5,
 9,
 1,
 0,
 3,
 2,
 3,
 3,
 9,
 1,
 7,
 6,
 2,
 3,
 2,
 2,
 5,
 0,
 7,
 4,
 9,
 7,
 8,
 3,
 2,
 1,
 1,
 8,
 3,
 6,
 1,
 0,
 2,
 1,
 0,
 0,
 1,
 9,
 2,
 7,
 3,
 0,
 4,
 6,
 5,
 2,
 6,
 4,
 7,
 2,
 8,
 9,
 9,
 3,
 0,
 7,
 1,
 0,
 2,
 0,
 3,
 5,
 4,
 6,
 5,
 8,
 6,
 3,
 7,
 3,
 8,
 0,
 9,
 1,
 0,


In [26]:
len(prediction[0])

60000

In [9]:
type(Y_train[0][0])

numpy.float64

In [10]:
Y_train.shape

(60000, 10)

In [11]:
X_train.shape

(60000, 784)

In [12]:
(X_train[0].shape)

(784,)

In [13]:
type(X_train)


numpy.ndarray

In [14]:
X_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [15]:
X_train[0].shape

(784,)

In [16]:
len(y_train)

60000

In [17]:
len(X_train)

60000

In [18]:
len(temp)


1

In [19]:
temp.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
temp[0][0]